In [1]:
import ollama
import os
import requests
import pandas as pd
import json
# import sys
import fiona

#functions:
from flood_functions import get_svi_stats_and_tracts, get_flash_flood_warnings, get_flood_data, get_flood_map, chat_function


#signatures:
signatures_path = "C:\\Users\\kimia\\OneDrive\\Documents\\Python Scripts\\FloodLlama\\signatures.txt"



#question = "What are the current flood alerts for the New Orleans area? "
# question= """For the "Lakeview neighborhood in New Orleans which is Located near Lake Pontchartrain", can you provide SVI statistics for Household Characteristics Theme with an SVI score smaller than 0.5?"""
#question ="""Is there any possible flood alert in Florida?"""
# question = """when was the last time we had flood in San Antonio,Texas?"""
# question = "When was the last time we had flood in Middlesex County, New Jersey?" 
question= "For Middlesex County in Massachusetts please provide SVI statistics for Household Characteristics Theme with an SVI score smaller than 0.5"

Instruction = """\n\n use the below function signutres and provide answer so that I can use the correct function.\n\n"""


# Read the content of the signatures.txt file
with open(signatures_path, 'r',encoding= "utf8") as file:
    prompt = file.read()

prompt2 = f"[INST] " + Instruction 
prompt += f"""{question}[/INST]"""

#test1
#print(prompt)

#test2(functions_calling_SVI)
# print(get_svi_stats_and_tracts("NJ","Middlesex County","RPL_THEME4",">",0.08))

#test3(function_calling_warning)
# print(get_flash_flood_warnings("FL"))

# test3(function_calling_data)
# print(get_flood_data("1130 Jaguar Pkwy San Antonio TX 78224"))
#94 Bayard St New Brunswick NJ 08901


In [2]:
print(prompt)

"""<FUNCTIONS>{
    "function": "get_svi_stats_and_tracts",
    "description": "Retrieve Social Vulnerability Index (SVI) statistics based on user-specified criteria and display census tract on map with SVI index. State abbreviation, county, theme,                     operation and threshold must always be specified.",
    "arguments": [
        {
            "name": "state_abbr",
            "type": "string",
            "description": "The state abbreviation (e.g., 'NJ')."
        },
        {
            "name": "county",
            "type": "string",
            "description": "The county name (e.g., 'Middlesex')."
        },
        {
            "name": "theme",
            "type": "string",
            "description": "The SVI theme to analyze. Choose from the following options:                                  - RPL_THEME1 (SVI for Socioeconomic Status theme);                                  - RPL_THEME2 (SVI for Household Characteristics theme);                                

In [3]:
result = ollama.generate("llama2_function_calling_q3",prompt)
response = result['response']
sample_answer = response.strip()
print(response)
print("------------------")

{
    "function": "get_svi_stats_and_tracts",
    "arguments": {
        "state_abbr": "MA",
        "county": "Middlesex",
        "theme": "RPL_THEME2",
        "op": "<",
        "threshold": 0.5
    }
}
------------------


In [4]:

sample_answer_dict = eval(sample_answer)
function_name = sample_answer_dict['function']
arguments = list(sample_answer_dict['arguments'].values())
print(function_name)
print(arguments)
print("------------------------")

get_svi_stats_and_tracts
['MA', 'Middlesex', 'RPL_THEME2', '<', 0.5]
------------------------


In [5]:
result = eval(f"globals()['{function_name}'](*arguments)")
print(result)

{"Statistics":{"Total_Areas":1,"Max_SVI":0.062,"Min_SVI":0.062,"Average_SVI":0.062},"Census_Tracts":[{"FIPS":25017,"RPL_THEME2":0.062}]}


In [6]:
second_prompt = f"""please polish the following answer based on the following question, coherent and human readble question:[BOS]{question}[EOS],
***
Answer:[BOS]"""+result+"""[EOS]"""

In [8]:
final_result = ollama.generate('llama2_chat_q2', second_prompt)
print(final_result)

{'model': 'llama2_chat_q2', 'created_at': '2024-09-25T15:51:08.2836924Z', 'response': 'Please polish the answer by making it more coherent and human readable, also provide additional details and explanations for the SVI statistics provided.', 'done': True, 'done_reason': 'stop', 'context': [529, 29989, 2962, 29918, 6672, 29918, 333, 29989, 29958, 1792, 29966, 29989, 355, 29918, 6672, 29918, 333, 29989, 3238, 13, 30004, 13, 552, 559, 1248, 728, 278, 1494, 1234, 2729, 373, 278, 1494, 1139, 29892, 16165, 261, 296, 322, 5199, 1303, 569, 1139, 10834, 29933, 3267, 29962, 2831, 341, 2205, 793, 735, 5127, 297, 16167, 3113, 3867, 13955, 29902, 13964, 363, 5619, 8948, 26804, 6765, 498, 2004, 411, 385, 13955, 29902, 8158, 7968, 1135, 29871, 29900, 29889, 29945, 29961, 29923, 3267, 1402, 13, 17435, 13, 22550, 10834, 29933, 3267, 3199, 29908, 9513, 6765, 28819, 11536, 29918, 29909, 12588, 1115, 29896, 1699, 7976, 29918, 7597, 29902, 1115, 29900, 29889, 29900, 29953, 29906, 1699, 8140, 29918, 7597, 

In [10]:
# System message to set up the assistant's role
your_system_message = """
You are an assistant that provides flood information relevant to the user queries. You may receive \
queries about flood alerts, Social Vulnerability Indexes (SVI) for a location, flood alerts, or display \
maps to the request of the user. Interactive maps are always displayed to the user. \
Always try your best to answer in natural language easy to understand using the available function calls.
"""

# Initialize the original prompt with the system message
original_prompt = f"<s>[INST] <<SYS>>\n{your_system_message}\n<</SYS>>\n\n"

# Function to manage the conversation flow
def chat_function(role, message, previous_message=original_prompt):
    """
    Inputs:
    - previous_message: The message from the previous conversation (defaults to original system message)
    - role: The role generating the current message (e.g., 'user' or 'assistant')
    - message: The new message/question to add to the conversation
    
    Returns:
    - Updated conversation log with the new message and answer from the assistant
    """
    
    # Add the user message to the conversation
    if role == 'user':
        if '</s>' not in previous_message:  # First user message after system setup
            previous_message += f"{message} [/INST]"
        else:  # For continuing conversations
            previous_message += f"<s>[INST] {message} [/INST]"
    
    # Add the assistant's response to the conversation
    elif role == 'assistant':
        previous_message += f"{message}</s>"
    
    else:
        print("Error: Invalid role")
    
    return previous_message

# Example of generating a user and assistant conversation flow
conversation_log = original_prompt

# Simulating user message
user_message = "For Bexar County in Texas please provide SVI for Housing Type/Transportation theme with an SVI score smaller than 1"
conversation_log = chat_function('user', user_message, conversation_log)


# Print final conversation log
print(conversation_log)


<s>[INST] <<SYS>>

You are an assistant that provides flood information relevant to the user queries. You may receive queries about flood alerts, Social Vulnerability Indexes (SVI) for a location, flood alerts, or display maps to the request of the user. Interactive maps are always displayed to the user. Always try your best to answer in natural language easy to understand using the available function calls.

<</SYS>>

For Bexar County in Texas please provide SVI for Housing Type/Transportation theme with an SVI score smaller than 1 [/INST]


In [12]:
def call_function_based_on_query(query):
    if "SVI" in query:
        # Extract relevant details from the user message
        state_abbr = "TX"  # Example extraction
        county = "Bexar"  # Example extraction
        theme = "RPL_THEME2"  # Housing Type/Transportation theme
        op = "<"
        threshold = 1
        # Call the function
        result = get_svi_stats_and_tracts(state_abbr, county, theme, op, threshold)
        return result
    else:
        return "Sorry, I couldn't find a relevant function for your query."

# Update assistant response in the conversation based on the function call
assistant_response = call_function_based_on_query(user_message)
conversation_log = chat_function('assistant', assistant_response, conversation_log)

# Print the updated conversation log with assistant response
print(conversation_log)


<s>[INST] <<SYS>>

You are an assistant that provides flood information relevant to the user queries. You may receive queries about flood alerts, Social Vulnerability Indexes (SVI) for a location, flood alerts, or display maps to the request of the user. Interactive maps are always displayed to the user. Always try your best to answer in natural language easy to understand using the available function calls.

<</SYS>>

For Bexar County in Texas please provide SVI for Housing Type/Transportation theme with an SVI score smaller than 1 [/INST]{"Statistics":{"Total_Areas":1,"Max_SVI":0.8635,"Min_SVI":0.8635,"Average_SVI":0.8635},"Census_Tracts":[{"FIPS":48029,"RPL_THEME2":0.8635}]}</s>{"Statistics":{"Total_Areas":1,"Max_SVI":0.8635,"Min_SVI":0.8635,"Average_SVI":0.8635},"Census_Tracts":[{"FIPS":48029,"RPL_THEME2":0.8635}]}</s>
